In [12]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
from PyPDF2 import PdfMerger
import matplotlib.pyplot as plt
sns.set_theme()

In [ ]:
TYPE = 'minions'

In [13]:
PATH_TO_PICS = f'reworked/{TYPE}'     #path to folder of the pictures
images = [os.path.join(PATH_TO_PICS, f) for f in os.listdir(PATH_TO_PICS)][4:8]
backs = []
for i in range(4):
    backs.append('000.png')
images, backs

(['reworked/spells\\Burger-@ Pittya(12-10-2022-10_16_56).png',
  'reworked/spells\\Cbw-Climbing(6-10-2022-15_3_43).png',
  'reworked/spells\\Cyberwarriorz(3-10-2022-9_32_52).png',
  'reworked/spells\\Get-Sniped(3-10-2022-13_58_17).png'],
 ['000.png', '000.png', '000.png', '000.png'])

In [27]:
width, height = int(8.27 * 300), int(11.7 * 300) # A4 at 300dpi
orig_w, orig_h = 785, 1100                       #original size of the pictures
sf = 1.5 * 0.92                                  #scaling factor
back_w, back_h = 1177, 1650                      #desired size of the cards calculated with the scaling factor 
minion_w, minion_h = 1095, 1535
pic_in_row, pic_in_col = int(width/back_w), int(height/back_h)     #number of pics in each row and col with the new size    
n = pic_in_row * pic_in_col

print(f'Page size: {width} {height}')
print(f'Num of pics in each row each col: {pic_in_row} {pic_in_col}')
print(f'Back width, back height: {back_w} {back_h}', )
print(f'Front width, back height: {minion_w} {minion_h}')

Page size: 2481 3510
Num of pics in each row each col: 2 2
Back width, back height: 1177 1650
Front width, back height: 1095 1535


In [29]:
#generate position of the pictures
back_boxes = []
front_boxes = []
for j in range(pic_in_row):
    for i in range(pic_in_col):
        back_boxes.append([i * back_w + 20,j*back_h + 20])
        front_boxes.append([i * minion_w + 20 + 50,j*minion_h + 35])
        
        
        
print(back_boxes)
print(front_boxes)        

front_boxes[1][0] = back_boxes[1][0] + 50
front_boxes[3][0] = back_boxes[3][0] + 50
front_boxes[2][1] = back_boxes[2][1] + 15
front_boxes[3][1] = back_boxes[3][1] + 15

print(back_boxes)
print(front_boxes)       

[[20, 20], [1197, 20], [20, 1670], [1197, 1670]]
[[70, 35], [1165, 35], [70, 1570], [1165, 1570]]
[[20, 20], [1197, 20], [20, 1670], [1197, 1670]]
[[70, 35], [1247, 35], [70, 1685], [1247, 1685]]


In [30]:
back_groups = [backs[i:i+n] for i in range(0, len(backs), n)]
front_groups = [images[i:i+n] for i in range(0, len(images), n)]

In [20]:
for i, group in enumerate(front_groups):
    page = Image.new('RGB', (width, height), 'white')                #create a blank page
    for idx,k in enumerate(group):
        try:
            rgba = np.array(Image.open(k))                          #read the image
            rgba[rgba[...,-1]==0] = [255,255,255,255]                 #transparent part to white
            curr_pic = Image.fromarray(rgba).resize((minion_w, minion_h), Image.ANTIALIAS) #resize pics, interpol
            page.paste(curr_pic, box=front_boxes[idx]),                   #put pics to the blank bage at the given location
        except IndexError:
            print('No more pic.')
        except Exception as e:
             print(e)
    
page.save('pages/page{}.pdf'.format(i))                               #save pic
    



In [21]:
for i, group in enumerate(back_groups):
    page = Image.new('RGB', (width, height), 'white')                #create a blank page
    for idx,k in enumerate(group):
        try:
            rgba = np.array(Image.open(k))                          #read the image
            rgba[rgba[...,-1]==0] = [255,255,255,255]                 #transparent part to white
            curr_pic = Image.fromarray(rgba).resize((back_w, back_h), Image.ANTIALIAS) #resize pics, interpol
            page.paste(curr_pic, box=back_boxes[idx]),                   #put pics to the blank bage at the given location
        except IndexError:
            print('No more pic.')
        except Exception as e:
             print(e)
    
page.save('pages/page{}.pdf'.format(i+1))                               #save pic

In [22]:
page_num = len(os.listdir('pages'))
pdfs = ['pages/page' + str(i) + '.pdf' for i in range(page_num)]

merger = PdfMerger()

for pdf in pdfs:
    merger.append(pdf)

merger.write("test_gerinek_3.pdf")
merger.close()